In [ ]:
# Alex Sessums & Valentin Lorenzen

In [1]:
# Data Analysis
import numpy as np
import pandas as pd

# Miscellaneous
import os

In [ ]:
path = '/Users/asessums/Desktop/valex'

In [5]:
# Load Data from Text File into Dataframe
df = pd.read_csv('data/Friday.csv', sep=';')
df.head(6)

,timestamp,customer_no,location
0,2019-09-06 07:00:00,1,dairy
1,2019-09-06 07:00:00,2,drinks
2,2019-09-06 07:00:00,3,fruit
3,2019-09-06 07:01:00,2,checkout
4,2019-09-06 07:01:00,4,drinks
5,2019-09-06 07:01:00,5,dairy
